In [1]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 4
seed = 69105
n_classifiers = 10
delta=0.05 ## what would this be?   
binary=True
bound='germain'
epsilons=[0.01]
alphas=[0.0, 0.1]#0,0.3]
sigmas=[[3,2],[1,2],[3,3]]

project_folder = "/cephyr/users/adambre/Alvis/"

# Alvis params

In [2]:
project = 'SNIC2021-7-83'
username = 'adambre'
job = 'batch_bound_single.sbatch'

In [3]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            105661     alvis     bash  adambre  R       9:08      1 alvis1-02 


In [4]:
!cat $job

#!/usr/bin/bash

#SBATCH -t 1-00:00:00
#SBATCH -N 1 --gpus-per-node=T4:1
#SBATCH -p alvis 

if [ -z "$task" ]
then
    task=2
fi
if [ -z "$seed" ]
then 
    seed=69105
fi
if [ -z "$alpha" ]
then 
    alpha=0.0
fi
if [ -z "$sigma" ]
then 
    sigma='3,3'
fi
if [ -z "$epsilon" ]
then 
    epsilon=0.01
fi
if [ -z "$delta" ]
then 
    delta=0.05
fi
if [ -z "$binary" ]
then 
    binary=0
fi
if [ -z "$nclassifiers" ]
then 
    nclassifiers=2
fi
if [ -z "$bound" ]
then 
    bound='germain'
fi
if [ -z "$prior" ]
then 
    prior=''
fi
if [ -z "$posterior" ]
then 
    posterior=''
fi

. load_modules.sh

python batch_bound_single.py -t $task -r $seed -a $alpha -s $sigma -e $epsilon -d $delta -b $binary -n $nclassifiers -B $bound -p $prior -P $posterior

In [7]:
print('Iterating over experiments...\n')

fids = []

os.makedirs('logs', exist_ok=True)


np.random.seed(seed)
for alpha in alphas:
    
    print("alpha:"+str(alpha))
    
    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("    sigma:"+str(sigma))
            arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                    epsilon=epsilon, binary=binary, n_classifiers=n_classifiers)
            for a in arg_list:   
                
                ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                fid = './logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                    (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)
                
                sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                        +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'])
                
                print(exp)
                prior_path = a['prior_path']
                #output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                jobid = int(output[0].split(' ')[-1])
                fid = fid+'-%s' % jobid

                fids.append(fid)

Iterating over experiments...

alpha:0.0
  epsilon:0.01
    sigma:[3, 2]


FileNotFoundError: [Errno 2] No such file or directory: 'posteriors/task2/Binary/10_0/params.txt'

In [5]:
print(fids)

NameError: name 'fids' is not defined

In [151]:
#print('scancel ' + ' '.join([str(i) for i in range(104783, 104841)]))

In [1]:
!squeue -u $username

squeue: option requires an argument -- 'u'
Try "squeue --help" for more information


In [159]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 50 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')

------
./logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_0-104870
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104870&from=1632907696000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_0.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]
Mak

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_180-104874
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104874&from=1632907697000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_180.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_360-104878
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104878&from=1632907697000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_360.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_540-104882
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104882&from=1632907696000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_540.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_4-104886
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104886&from=1632907697000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/2_4.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]

...

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_0-104898
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104898&from=1632907699000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '1.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_0.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_180-104902
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104902&from=1632907698000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '1.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_180.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_360-104906
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104906&from=1632907698000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '1.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_360.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

 


------
./logs/batch_t-2_r-69105_a-0.0000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_540-104910
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104910&from=1632907701000
{'task': 2, 'seed': 69105, 'alpha': 0.0, 'sigma': '1.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 10, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/0/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_0/1_540.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.00258

UnboundLocalError: local variable 'child' referenced before assignment